In [1]:
!pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import selenium
from selenium import webdriver
import time
import math
import bokeh
import matplotlib.pyplot as plt

In [3]:
columns=['刊登者','房型','標題','價格','地址','瀏覽數','收藏數','桌子', '椅子', '衣櫃', '床', '冰箱', '冷氣', '洗衣機', '網路', '沙發', '熱水器', '天然瓦斯', '電視',
 '第四台','網址']

In [ ]:
def get_data(urls):
    data=[]
    for url in urls:
        try:
            resp=requests.get(url)
            soup=BeautifulSoup(resp.text,'lxml')
            temp=[]

            user_id=soup.find('input',id="hid_user_id")['value']
            house_type=soup.find('div',id="propNav").find_all('a')[4].text
            title=soup.find('title').text
            price=int(soup.find('div',class_="price clearfix").i.text.split(' ')[0].replace(',',''))
            view_count=int(soup.find('div',class_="pageView clearfix").find('b').text)
            address=soup.find('span',class_="addr").text
            collect_num=int(re.search(r'\d+',soup.find('li',class_="collect").text).group())
            temp.append(user_id)
            temp.append(house_type)
            temp.append(title)
            temp.append(price)
            temp.append(address)
            temp.append(view_count)
            temp.append(collect_num)


            #房東提供
            try:
                itemInHouse=soup.find('ul',class_="facility clearfix").find_all('li')
                #haveItem=[]
                for i in itemInHouse:
                    if 'no' not in i.span['class']:
                        temp.append(1)
                    else:
                        temp.append(0)
            except:
                for i in range(13):
                    temp.append(None)
            temp.append(resp.url)
            data.append(temp)
        except:
            pass
    return data

In [ ]:
country_name='高雄'
town_name=['三民','小港','大社']
#最多五個鄉鎮

In [ ]:
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values' :
        {
        'notifications' : 2
         }
}
urls=[]
options.add_experimental_option('prefs',prefs)
driver = webdriver.Chrome(options=options)
driver.get('https://rent.591.com.tw/?kind=0&region=1')

for country in driver.find_element_by_class_name('area-box-body').find_elements_by_tag_name('dd'):
    if country_name in country.text:
        country.click()
        break
time.sleep(2)
driver.find_element_by_id('search-location').find_elements_by_tag_name('span')[1].click()
for i in driver.find_element_by_id('optionBox').find_elements_by_tag_name('label'):
    for j in town_name:
        if j in i.text:
            i.click()

In [ ]:
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values' :
        {
        'notifications' : 2
         }
}
urls=[]
options.add_experimental_option('prefs',prefs)
driver = webdriver.Chrome(options=options)
driver.get('https://rent.591.com.tw/?kind=0&region=1')

for country in driver.find_element_by_class_name('area-box-body').find_elements_by_tag_name('dd'):
    if country_name in country.text:
        country.click()
        break
time.sleep(2)
driver.find_element_by_id('search-location').find_elements_by_tag_name('span')[1].click()
for i in driver.find_element_by_id('optionBox').find_elements_by_tag_name('label'):
    for j in town_name:
        if j in i.text:
            i.click()
time.sleep(3)
pageNum=driver.find_element_by_xpath('//*[@id="container"]/section[5]/div/div[1]/div[3]/div[1]/i').text
if ',' in pageNum:
    pageNum=int(pageNum.replace(',',''))
page=math.ceil(int(pageNum)/30)
try:
    driver.find_element_by_class_name('area-box-close').click()
except:
    pass

for i in range(page-1):
    res=driver.page_source
    soup=BeautifulSoup(res,'lxml')
    for i in soup.find_all('h3'):
        urls.append('https:'+i.find('a')['href'].strip())
    try:
        time.sleep(1)
        driver.find_element_by_link_text('下一頁').click()
    except:
        print('no page can crawl')
driver.close()
print('正在抓取資料，共%d頁' % page)
data=get_data(urls)
print('抓取完成')

In [ ]:
table=pd.DataFrame(data=data,columns=columns)
table.shape

In [ ]:
#模組1:比較不同房型的價格區間
#table.boxplot(column='價格',by='房型')
def price_range(table,town=None):
    if town!= None:
        table=table[table['地址'].str.contains(town)]
    plt.subplots_adjust(wspace =10, hspace =0)
    plt.figure(figsize=(20,5))
    plt.subplot(1,4,1)
    #plt.title('分租套房')
    plt.boxplot(x=table['價格'][table['房型']=='分租套房'].to_list())
    plt.subplot(1,4,2)
    #plt.title('獨立套房')
    plt.boxplot(table['價格'][table['房型']=='獨立套房'].to_list())
    plt.subplot(1,4,3)
    #plt.title('雅房')
    plt.boxplot(table['價格'][table['房型']=='雅房'].to_list())
    plt.subplot(1,4,4)
    #plt.title('整層住家')
    plt.boxplot(table['價格'][table['房型']=='整層住家'].to_list())
    #不同房型的數量
price_range(table,town=town_name[1])

In [ ]:
#模組2
item=['桌子', '椅子', '衣櫃', '床', '冰箱', '冷氣', '洗衣機', '網路', '沙發', '熱水器', '天然瓦斯', '電視','第四台']
table=table.dropna()
base=len(item)
table['設備完善率']=table.loc[:,item].sum(1)/base
#plt.boxplot(table['設備完善率'])
plt.hist(table['設備完善率'],edgecolor='black')

In [ ]:
#模組3:熱門物件
from bokeh.plotting import figure,output_file,show
from bokeh.models import CategoricalColorMapper

map=CategoricalColorMapper(factors=['分租套房','獨立套房','雅房','整層住家'],palette=['red','orange','blue','green'])
p=figure(title='不同物件的瀏覽數與收藏數')
p.circle(x='收藏數',y='瀏覽數',source=table,size=10,color={'field':'房型','transform':map})
show(p)

In [ ]:
#模組4:搭配前面的所有模組，選出具備理想條件的刊登者的所有物件
equiment=0.8
area=town_name[0]#'士林區'
price_limit=[9000,12000]
house_type='套房'
table4=table[(table['設備完善率']>=equiment)&(table['地址'].str.contains(area))&(table['價格']>=price_limit[0])&(table['價格']<=price_limit[1])&(table['房型'].str.contains(house_type))]
select_user=list(set(table4['刊登者']))
table5=table[table['刊登者'].isin(select_user)]
print('共有 %d 筆符合的資料' % table5.shape[0])
table5

In [ ]:
table4